In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from datassist import explore

# Inital exploration

In [ ]:
df = pd.read_csv('data/')
df.head()

In [ ]:
explore.overview(df)

In [ ]:
explore.basic_stats(df)